# Our First Use Case: Flask API and Service Layer

* Learn to differentiate orchestration logic, business logic, and interfacing code
* Introduce *Service Layer* pattern

## Connecting Our Application To The Real World

* We have the core of our domain model, and we have the repository pattern for permanent storage.
* Let's put together the moving parts as quickly as possible and then refactor
* Plan:
    1. Use `Flask` to put an API endpoint in front of our `allocate` domain service.
Wire up the database session and our repo. Test it with an end-to-end test and some quick SQL to prepare
test data.
    1. Refactor our a service layer that can serve as an abstraction to capture the use case and that will sit between Flask and our domain model. Build some service layer tests that use `FakeRepository`.
    1. Experiment with different types of parameters for our service layer functions.
Show that using primitive data types allows the service layer's clients (tests and Flask) to be decoupled from the model layer.

## A First End-To-End Test

In [2]:
import pytest

@pytest.mark.usefixtures("restart_api")
def test_api_returns_allocation(add_stock):
    sku, other_sku = random_sku(), random_sku("other")
    early_batch = random_batchref(1)
    later_batch = random_batchref(2)
    other_batch = random_batchref(3)
    add_stock([
        (later_batch, sku, 100, "2022-01-17"),
        (early_batch, sku, 100, "2022-01-15"),
        (other_batch, othersku, 100, None),
    ])
    data = {"orderid": random_orderid(), "sku": sku, "quantity": 3}
    url = config.get_api_url()
    r = requests.post(f"{url}/allocate", json=data)
    assert r.status_code == 201
    assert r.json()["batchref"] == earlybatch

## The Straightforward Implementation

In [4]:
from flask import Flask

app = Flask(__name__)

@app.route("/allocate", methods=["POST"])
def allocate_endpoint():
    session = get_session()
    batches = repository.SqlAlchemyRepository(session).list()
    line = model.OrderLine(
        request.json["orderid"],
        request.json["sku"],
        request.json["quantity"],
    )
    batchref = model.allocate(line, batches)
    
    return jsonify({"batchref": batchref}), 201

* This doesn't actually save our allocation to the database, but our first test will pass since it doesn't check that!
* Need to write a new test to cover this:
    * Inspect database state (not black-boxy)
    * Can't allocate a second line if the first depleted the batch

In [5]:
@pytest.mark.usefixtures("restart_api")
def test_allocations_are_persisted(add_stock):
    sku = random_sku()
    batch1, batch2 = random_batchref(1), random_batchref(2)
    order1, order2 = random_orderid(1), random_orderid(2)
    add_stock([
        (batch1, sku, 10, "2022-01-17"),
        (batch2, sku, 10, "2022-01-18"),
    ])
    line1 = {"orderid": order1, "sku": sku, "quantity": 10}
    line2 = {"orderid": order2, "sku": sku, "quantity": 10}
    url = config.get_api_url()
    
    # First order uses up all stock in batch 1
    r = requests.post(f"{url}/allocate", json=line1)
    assert r.status_code == 201
    assert r.json()["batchref"] == batch1
    
    # Second order should go to batch 2
    r = requests.post(f"{url}/allocate", json=line2)
    assert r.status_code == 201
    assert r.json()["batchref"] == batch2

## Error Conditions That Require Database Checks

* This is an ugly test, and our code will only get uglier as we add more of these tests
* What if the domain raises an error for an SKU that is out of stock? Or an SKU that doesn't exist?
    * These are not things the domain should know about
    * These should be at the database level before we hit the domain.
    * Testing these would require more e2e tests, which starts to get out of control
    * This is called an *inverted testing pyramid*
        * We want to have most of the tests at the smallest levels of the code. From most desirable to least:
            1. Unit Tests
            1. Integration Tests
            1. E2E Tests
    
## Introducing A Service Layer, And Using FakeRepository To Unit Test It

* Our Flask app, which should be doing only web stuff like parsing json, is also doing things like checking the database state, handling errors, and updating the database when it should be updated.
* The latter stuff is called *orchestration*, *use-cases*, or *services*
    * These are an abstraction that sit between the repository and the domain model
    * These don't really need to be tested by e2e tests
    * You only really need these when orchestration logic starts to seep into the repository or domain
* These services are split out into a *Service Layer*/*Orchestration Layer*/*Use-Case Layer*

In [6]:
class FakeRepository:
    def __init__(self, batches):
        self._batches = set(batches)
        
    def add(self, batch):
        self._batches.add(batch)
        
    def get(self, reference):
        return next(b for b in self._batches if b.reference == reference)
    
    def list(self):
        return list(self._batches)
    
class FakeSession:
    committed = False
    
    def commit(self):
        self.committed = True
    
def test_returns_allocations():
    line = model.OrderLine("o1", "chair", 10)
    batch = model.Batch("b1", "chair", 100)
    repo = FakeRepository(batches=[batch])
    
    result = services.allocate(line, repo, FakeSession())  # service.allocate != model.allocate
    assert result == "b1"

* Much cleaner! We can now test this functionality without dealing with the Flask API and an actual database.
* Our Flask app would call `services.allocate` to deal with the database and domain stuff. It would now only explicitly implement web stuff.

## Why Is Everything Called A Service?

* What is the difference between a *domain service* and a *service layer*?
    * *Domain service* - a piece of logic that belongs to the domain but doesn't sit inside a stateful entity.
    * *Service layer* - orchestrates some operation based on requests from the outside
    
## Wrap-Up
  
* Pros:
    * Use cases in one location
    * Our domain logic remains veiled, making it easier to refactor
    * Separation of stuff that takes HTTPS from stuff that talks allocation
    * When combined with the Repository Pattern, allows us to unit test rather than integration or E2E test.
* Cons:
    * If the app is purely a web app, controllers/views can capture all the use cases
    * Adds another layer of abstraction that can complicate things for newer devs
    * Putting too much logic into the service layer leads to the [Anemic Domain anti-pattern](https://en.wikipedia.org/wiki/Anemic_domain_model).
    * You could push this logic down to your domain models without needing the extra layer in between. This is known as "fat models, thin controllers"